# Practical Work 3


## Session 2: Validation of the results

### Lluis Pellicer Juan y Jorge De la Cruz Martínez

In [1]:
import sys
sys.path.append(r'C:/Users/JUAN LENOVO/Desktop/3 GCD/2º cuatrimestre/LNR/PRÁCTICA 9/TransformersTuning')


import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datareader import PAN24Reader
from fine_tuning import training as fine_tuning_training, validate as fine_tuning_validate
from fine_tuning_PEFT import training as fine_tuning_PEFT_training, validate as fine_tuning_PEFT_validate
from mydataset import MyDataset
from utils import set_seed

def train_and_evaluate(model_name, dataset, training_args, compute_metrics):
    # Add code here to load dataset and initialize tokenizer
    myReader = PAN24Reader()
    df = myReader.load_dataset_classification(dataset, string_labels=False, positive_class='conspiracy')
    X_train, X_val = train_test_split(df, test_size=0.1, random_state=SEED, shuffle=True, stratify=df['label'])
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    
    # Fine-tuning
    # Fine-tuning
    model = fine_tuning_training(model, X_train, X_val, training_args.learning_rate, 
                                  training_args.optimizer_name, training_args.schedule, training_args.epochs,
                                  tokenizer, batch_size=training_args.batch_size, padding="max_length")



    # Evaluation
    results = fine_tuning_validate(model, dataset, tokenizer, padding="max_length", max_length=512)
    print("Results:", results)
    return results



dataset_en = "C:/Users/JUAN LENOVO/Desktop/3 GCD/2º cuatrimestre/LNR/PRÁCTICA 9/TransformersTuning/dataset_oppositional/training/dataset_oppositional/dataset_en_train.json"
dataset_es = "C:/Users/JUAN LENOVO/Desktop/3 GCD/2º cuatrimestre/LNR/PRÁCTICA 9/TransformersTuning/dataset_oppositional/training/dataset_oppositional/dataset_es_train.json"


# Define training arguments
training_args = TrainingArguments(
    learning_rate=1e-5,
    optimizer_name="adam",
    schedule="linear",
    epochs=5,
    batch_size=32,
    patience=3,
    measure="accuracy",
    out="./out"
)

# Define compute_metrics function if needed

# Set seed for reproducibility
set_seed(1234)

# Train and evaluate model on English dataset
results_en = train_and_evaluate(model_name, dataset_en, training_args, compute_metrics)
print("Results in English:", results_en)

# Train and evaluate model on Spanish dataset
results_es = train_and_evaluate(model_name, dataset_es, training_args, compute_metrics)
print("Results in Spanish:", results_es)

# Cross-validation for English dataset
avg_mcc_en, avg_f1_en = cross_validation(dataset_en, model_name, training_args, compute_metrics)
print("Average MCC for English (cross-validation):", avg_mcc_en)
print("Average F1-score for English (cross-validation):", avg_f1_en)

# Cross-validation for Spanish dataset
avg_mcc_es, avg_f1_es = cross_validation(dataset_es, model_name, training_args, compute_metrics)
print("Average MCC for Spanish (cross-validation):", avg_mcc_es)
print("Average F1-score for Spanish (cross-validation):", avg_f1_es)


ModuleNotFoundError: No module named 'torch'